In [2]:
import pytesseract
from PIL import Image
import gradio as gr
import os


In [3]:
# Function to extract text from an image (supports both Hindi and English)
def extract_text(image_path):
    try:
        img = Image.open(image_path)
        text = pytesseract.image_to_string(img, lang='eng+hin')
        return text
    except Exception as e:
        return str(e)

# Test with a sample image
# extracted_text = extract_text('sample_image.png')  # Use any image file path
# print(extracted_text)


In [4]:
# Function to search for a keyword in the extracted text
def search_text(extracted_text, keyword):
    matching_lines = [line for line in extracted_text.split('\n') if keyword.lower() in line.lower()]
    return "\n".join(matching_lines) if matching_lines else "No matches found."


In [5]:
import gradio as gr

# Function to process the uploaded image and extract text
def process_image(image):
    extracted_text = extract_text(image)
    return extracted_text

# Function to search for a keyword within the extracted text
def search_keyword(extracted_text, keyword):
    if keyword.lower() in extracted_text.lower():
        # Highlighting matching text
        result = [line for line in extracted_text.split('\n') if keyword.lower() in line.lower()]
        return "\n".join(result)
    else:
        return "No matching text found."

# Gradio interface
def ocr_search_app(image, keyword):
    extracted_text = process_image(image)
    search_result = search_keyword(extracted_text, keyword)
    return extracted_text, search_result

# Gradio UI components
interface = gr.Interface(
    fn=ocr_search_app, 
    inputs=[gr.Image(type="filepath", label="Upload Image"), gr.Textbox(label="Keyword to search")],
    outputs=[gr.Textbox(label="Extracted Text"), gr.Textbox(label="Search Results")]
)

# Launch the application
interface.launch()

Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


In [6]:
import cv2
print(cv2.__version__)


4.10.0


In [7]:
pip install pytesseract opencv-python numpy pillow


Note: you may need to restart the kernel to use updated packages.


In [8]:
def preprocess_image(image_path):
    # Load the image
    image = cv2.imread(image_path)

    # Check if the image was loaded successfully
    if image is None:
        raise FileNotFoundError(f"Image file '{image_path}' not found or could not be loaded.")

    # Convert the image to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Apply Gaussian blur to reduce noise and improve OCR accuracy
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)

    # Apply adaptive thresholding to create a binary image
    binary = cv2.adaptiveThreshold(blurred, 255,
                                   cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
                                   cv2.THRESH_BINARY, 11, 2)

    return binary


In [10]:
import pytesseract
import gradio as gr
from PIL import Image
import re

# Function to process the uploaded image and extract text using PyTesseract
def process_image(image):
    if image is not None:
        extracted_text = pytesseract.image_to_string(image, lang='eng+hin')  # For Hindi and English
        return extracted_text
    else:
        return "No image provided."

# Function to search for keywords within the extracted text and highlight matches
def search_keyword(extracted_text, keyword):
    if keyword:
        highlighted_text = re.sub(f"({keyword})", r'**\1**', extracted_text, flags=re.IGNORECASE)
        if highlighted_text == extracted_text:  # No match found
            return "No matching text found."
        return highlighted_text
    return "No keyword provided."

# Main function to process OCR and keyword search together
def ocr_search_app(image, keyword):
    extracted_text = process_image(image)
    search_result = search_keyword(extracted_text, keyword)
    return extracted_text, search_result

# Build the Gradio interface
interface = gr.Interface(
    fn=ocr_search_app, 
    inputs=[
        gr.Image(type="pil", label="Upload Image"),  # "pil" to pass as PIL image
        gr.Textbox(label="Keyword to search")
    ],
    outputs=[
        gr.Textbox(label="Extracted Text"), 
        gr.Markdown(label="Search Results (Highlighted)")
    ],
    title="OCR & Keyword Search (Hindi + English)",
    description="Upload an image containing Hindi and English text, extract the text using PyTesseract, and search for specific keywords. Matching sections will be highlighted."
)

# Launch the Gradio app with share=True
interface.launch(share=True)


Running on local URL:  http://127.0.0.1:7861

Could not create share link. Please check your internet connection or our status page: https://status.gradio.app.
